In [1]:
%pip install --upgrade sagemaker

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 532 kB/s eta 0:00:01    |██████████████████▎             | 6.5 MB 3.6 MB/s eta 0:00:02
     |████████████████████████████████| 14.0 MB 612 kB/s eta 0:00:01    |█████████▎                      | 4.1 MB 3.3 MB/s eta 0:00:03     |█████████████████████████▎      | 11.0 MB 4.7 MB/s eta 0:00:01��█████████▌  | 12.9 MB 612 kB/s eta 0:00:02
     |████████████████████████████████| 78 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 394 kB 4.2 MB/s eta 0:00:01     |██████████████████▎             | 225 kB 4.2 MB/s eta 0:00:01


In [15]:
import sagemaker
import os
import boto3
from botocore.config import Config
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session.region_name
bucket = sess.default_bucket()
prefix = 'octo/datasets/aloha_sim_dataset'
base_job_name = 'octo-flax'
py_version = "py310"
source_dir = f"{os.getcwd()}"
entry_point = "02_finetune_new_observation_action.py"
instance_type = "ml.g5.48xlarge"
instance_count = 1
keep_alive_period_in_seconds = 300
image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/tensorflow-training:2.14.1-gpu-py310-cu118-ubuntu20.04-sagemaker"
s3_client = boto3.client('s3')

## Upload the Octo finetuning dataset to Amazon S3

In [7]:
!curl https://rail.eecs.berkeley.edu/datasets/example_sim_data.zip -O
!unzip example_sim_data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  399M  100  399M    0     0   441k      0  0:15:26  0:15:26 --:--:--  783k     0  0:17:53  0:00:38  0:17:15  411k  0:00:48  0:16:48  421k9k      0  0:17:31  0:00:50  0:16:41  424k01:02  0:16:21  404k:01:09  0:16:08  411k  0:17:09  0:01:15  0:15:54  426k  0   397k      0  0:17:10  0:01:21  0:15:49  388k:18:06  0:01:47  0:16:19  340kk      0  0:18:23  0:02:03  0:16:20  376k 0:02:04  0:16:16  407k16:01  348k0  0:18:08  0:02:29  0:15:39  450k02:30  0:15:40  421k2  0:15:41  368k   0  0:18:11  0:02:41  0:15:30  426k5  0:03:01  0:15:04  401k 395k14:33  380k    0  0:17:48  0:04:06  0:13:42  402k   0  0:17:47  0:04:10  0:13:37  394k 0:04:33  0:13:08  443k0     0   385k      0  0:17:40  0:04:38  0:13:02  391k42  0:12:57  419k9  420k0:05:15  0:12:11  417k  0:17:24  0:05:48  0:11:36  405k     0  0:17:24  0:05:54  0:11:30  365k0  0:17:23  

In [8]:
local_directory = f"{os.getcwd()}/aloha_sim_dataset"

for root, dirs, files in os.walk(local_directory):

    for filename in files:

        local_path = os.path.join(root, filename)
        relative_path = os.path.relpath(local_path, local_directory)
        s3_path = os.path.join(prefix, relative_path)
    
        print('Searching "%s" in "%s"' % (s3_path, bucket))
        try:
            s3_client.head_object(Bucket=bucket, Key=s3_path)
            print(f"Path found on S3! Skipping {s3_path}...")
        except:
            print("Uploading %s..." % s3_path)
            s3_client.upload_file(local_path, bucket, s3_path)

Searching "octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/dataset_statistics_efc466d3677b6697010485135012e2cd326145c80d09a51884ea8856801ebfb0.json" in "sagemaker-us-west-2-097242523850"
Uploading octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/dataset_statistics_efc466d3677b6697010485135012e2cd326145c80d09a51884ea8856801ebfb0.json...
Searching "octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/features.json" in "sagemaker-us-west-2-097242523850"
Uploading octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/features.json...
Searching "octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/aloha_sim_cube_scripted_dataset-train.tfrecord-00006-of-00008" in "sagemaker-us-west-2-097242523850"
Uploading octo/datasets/aloha_sim_dataset/aloha_sim_cube_scripted_dataset/1.0.0/aloha_sim_cube_scripted_dataset-train.tfrecord-00006-of-00008...
Searching "octo/datasets/aloha_sim_dataset/aloha_sim_cube_scr

In [12]:
!rm example_sim_data.zip
!rm -fr aloha_sim_dataset

## Finetune Octo on Amazon SageMaker

In [20]:
flax_estimator = TensorFlow(
    role = role,
    instance_count = instance_count,
    base_job_name = base_job_name,
    image_uri = image_uri,
    py_version = py_version,
    source_dir = source_dir,
    entry_point = entry_point,
    instance_type = instance_type,
    keep_alive_period_in_seconds = keep_alive_period_in_seconds,
)

flax_estimator.fit(inputs={
    'train': f"s3://{bucket}/{prefix}"})

INFO:sagemaker:Creating training-job with name: octo-flax-2024-07-23-09-29-36-230


2024-07-23 09:30:04 Starting - Starting the training job...
2024-07-23 09:30:05 Pending - Training job waiting for capacity...........................................................................................................................................................................